jax배열 만들기

In [2]:
import jax.numpy as jnp

In [3]:
#넘파이 배열 선언하기
#배열은 넘파이와 똑같다
a=jnp.array([[1,2,3],[4,5,6]])
print(a)

[[1 2 3]
 [4 5 6]]


스칼라배

In [4]:
print(a*10)

[[10 20 30]
 [40 50 60]]


In [5]:
b=jnp.array([3,6,9])

In [6]:
#브로드 캐스트
print(a*b)

[[ 3 12 27]
 [12 30 54]]


원소 접근

In [8]:
print(a[0])
print(a[1][2])

[1 2 3]
6


In [9]:
for row in a:
  print(row)

[1 2 3]
[4 5 6]


In [10]:
#평탄화
c=a.flatten()
print(c)

[1 2 3 4 5 6]


퍼셉트론 구현

In [12]:
def AND(x1,x2) -> float:
  w1,w2,theta = 1.0,1.0,1.5
  tmp=x1*w1+x2*w2
  if tmp<=theta:
    return 0
  else:
    return 1

In [14]:
AND(1,0)

0

In [18]:
def AND2(x1,x2) -> float:
  x=jnp.array([x1,x2])
  w=jnp.array([1,1])
  b=-0.7
  tmp=jnp.sum(x*w)+b
  if tmp<=0:
    return 0
  else:
    return 1

In [19]:
AND2(1,1)

1

신경망 구현